In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import os
import warnings
import pickle
from timeit import default_timer as timer
from collections import OrderedDict
from itertools import chain

import matplotlib.pyplot as plt
import plotly.graph_objects as ply_go
import plotly.figure_factory as ply_ff
import plotly.colors as ply_colors 

from scipy import signal
import librosa
import librosa.display as librosa_display
from utils.preprocessing import prepare_data


In [2]:
# !pip install --upgrade pip
# !pip install soundfile
# !pip install audioread

In [3]:
from utils.load_metadata import load_metadata

In [4]:
root = os.getcwd()
directory= '/dataset/subset_dataset/covid19-cough-audio-classification/'
metadata = load_metadata(root, directory, metadata_file="metadata_compiled.csv")

Index(['uuid', 'datetime', 'cough_detected', 'SNR', 'latitude', 'longitude',
       'age', 'gender', 'respiratory_condition', 'fever_muscle_pain', 'status',
       'quality_1', 'cough_type_1', 'dyspnea_1', 'wheezing_1', 'stridor_1',
       'choking_1', 'congestion_1', 'nothing_1', 'diagnosis_1', 'severity_1',
       'quality_2', 'cough_type_2', 'dyspnea_2', 'wheezing_2', 'stridor_2',
       'choking_2', 'congestion_2', 'nothing_2', 'diagnosis_2', 'severity_2',
       'quality_3', 'cough_type_3', 'dyspnea_3', 'wheezing_3', 'stridor_3',
       'choking_3', 'congestion_3', 'nothing_3', 'diagnosis_3', 'severity_3',
       'quality_4', 'cough_type_4', 'dyspnea_4', 'wheezing_4', 'stridor_4',
       'choking_4', 'congestion_4', 'nothing_4', 'diagnosis_4', 'severity_4'],
      dtype='object')
NULL or NA records for each column:
uuid                         0
datetime                     0
cough_detected               0
SNR                          0
latitude                 11466
longitude    

In [5]:
metadata['audio_class'] = 'X' # default, we should have none by the end of this classification process
metadata.loc[ (metadata['cough_detected'] >= 0.80) & (metadata['age']>=60) ,'audio_class'] = 'A'
metadata.loc[ (metadata['cough_detected'] >= 0.80) & (metadata['age']>=40) & (metadata['age']<60) ,'audio_class'] = 'B'
metadata.loc[ (metadata['cough_detected'] >= 0.80) & (metadata['age']< 40) ,'audio_class'] = 'C'
metadata.loc[ (metadata['cough_detected'] < 0.80) & (metadata['age']>=60) ,'audio_class'] = 'D'
metadata.loc[ (metadata['cough_detected'] < 0.80) & (metadata['age']>=40) & (metadata['age']<60) ,'audio_class'] = 'E'
metadata.loc[ (metadata['cough_detected'] < 0.80) & (metadata['age']< 40) ,'audio_class'] = 'F'

print("Entries subdivided in classes. Printing the number of entries for each class:")
print(metadata[['audio_class','uuid']].groupby(['audio_class']).count().rename(columns={'uuid':'N_entries'}) )

print("\n\n\nSplitting count by class and status:")
print(metadata[['audio_class','status','uuid']].groupby(['audio_class','status']).count().rename(columns={'uuid':'N_entries'}) )

Entries subdivided in classes. Printing the number of entries for each class:
             N_entries
audio_class           
A                  686
B                 3261
C                 6344
D                  511
E                 1523
F                 2893
X                12332



Splitting count by class and status:
                         N_entries
audio_class status                
A           COVID-19            33
            healthy            600
            symptomatic         53
B           COVID-19           158
            healthy           2561
            symptomatic        542
C           COVID-19           417
            healthy           4744
            symptomatic       1183
D           COVID-19            26
            healthy            431
            symptomatic         54
E           COVID-19           110
            healthy           1193
            symptomatic        220
F           COVID-19           221
            healthy           2203
          

In [6]:
# First filter for just COVID-19 and healthy status
filtered_metadata = metadata[metadata['status'].isin(['COVID-19', 'healthy'])]

# Process each class and balance the samples
#balanced_dfs = []
for iclass in ['A', 'B', 'C', 'D', 'E', 'F', 'X']:
    tmp_metadata = filtered_metadata[filtered_metadata['audio_class'] == iclass]
    
    # Skip if class has no samples
    if len(tmp_metadata) == 0:
        continue
        
    # Apply SNR filter
    tmp_metadata = tmp_metadata[tmp_metadata['SNR'] >= tmp_metadata['SNR'].quantile(0.10)]
    
    # Get minimum count and round up to nearest 10
    min_count = tmp_metadata['status'].value_counts().min()
    n_resampling = int(np.ceil(min_count/10)*10)
    
    # Balance the samples
    #balanced_class = sample_all_data(tmp_metadata, 'status', target_size)
    #balanced_dfs.append(balanced_class)

# Combine all balanced classes
final_metadata = pd.concat([tmp_metadata])

# Verify the results
print("\nFinal count by class and status:")
print(final_metadata[['audio_class', 'status', 'uuid']].groupby(['audio_class', 'status']).count())

###
my_sampling_rate = int(4096*2)  # Sampling rate, how frequently we want to take a value of the audio curve.
                                 # the max frequency in the STFT will be (approximately) half of this
                               
my_n_fft = 512 # number of frequency bins to be calculated in the STFT; 
               # if my_window_size is None, this drives also the time-sampling window
    
my_n_mfcc = 26 # number of mel-frequencies used for the MFCC calculation. The original article used
               # a number of 13 MFCC frequencies, I am trying to add some extra info   
    
my_window_size = None # should not be greater than n_fft
target_duration = 10 # seconds; shorter audios will be zero padded; longer audios will be cut;
                     # obtained from an earlier dry run over all data and charting the distribution 
                     # of duration of the raw sound samples; 10 sec corresponds to the 97th percentile and 
                     # represent a significant improvement in terms of computing time (x5 faster) 
                     # respect to more conservative choices like 70 seconds (99th percentile)

### these are the frequency bins used to compute short-term features as per the original article (used as inputs to the cough detection classifier).
### Note that the code here will compute features for all bins but in the original paper they use only non-contiguous values.
### For example, we keep the bin [0, 200] Hz but not the [200, 300] Hz; the lowest bin used inthe rest of htis analysis will be [0, 200] Hz,
### the highest one will be [3800, 3900] Hz
my_psd_freqs = [0.0, 200.0, 300.0, 425, 500.0, 650.0, 950.0, 1150.0, 1400.0, 1800.0, 2300.0, 2400.0, 2850.0, 2950.0, 3800.0, 3900, 4000]
psd_feature_names =['SPEC_CENTROID', 'SPEC_WIDTH', 'SPEC_CREST', 'SPEC_MEAN', 'SPEC_SD', 'SPEC_SKEW']


mfcc_feature_names = [ "MFCC_MEAN_{:02}".format(i) for i in range(0,my_n_mfcc,1)]
mfcc_feature_names = mfcc_feature_names + [ "MFCC_SD_{:02}".format(i) for i in range(0,my_n_mfcc,1)]

##
data_dir = root + directory

all_data, all_audio_metadata, spectrograms_data = prepare_data(input_data=tmp_metadata, audio_datadir=data_dir, sr=my_sampling_rate, 
                                            target_duration=target_duration, n_fft=my_n_fft, n_mfcc=my_n_mfcc, 
                                            fft_window_size=my_window_size, psd_freq_bins=my_psd_freqs,
                                            mfcc_feature_names=mfcc_feature_names, psd_feature_names=psd_feature_names,
                                            max_audio_samples=None, print_every_n=20, tmp_metadata = tmp_metadata) 

all_audio_metadata = all_audio_metadata.drop_duplicates(subset='UUID', keep='first') # this avoids spurious duplicates at the following merge

print("Merging dataframe with audio features and df with audio metadata")
all_data = pd.merge(all_data,all_audio_metadata,on=['UUID'],how='inner')
all_data.drop(['AUDIO_FILE'],axis=1,inplace=True)

#print(all_data[['UUID','STATUS']].groupby(['STATUS']).count())
print("Shape of full dataframe with features and labels: {}".format(all_data.shape))
all_data.head(10)


Final count by class and status:
                      uuid
audio_class status        
X           COVID-19   189
            healthy    745



/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecat

STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)


/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecat

STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)


...

STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)


/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecat

STFT DIMENSIONS:(257, 161)
STFT DIMENSIONS:(257, 161)
Merging dataframe with audio features and df with audio metadata
Shape of full dataframe with features and labels: (934, 115)


/Users/rodolfolobocarrasco/Documents/projects/ESPOL-PROJECT/ESPOL-coughvid-project/utils/preprocessing.py:17: UserWarning: PySoundFile failed. Trying audioread instead.
  t, sr = librosa.load(indir + filename, sr=sr, mono=True)
/Users/rodolfolobocarrasco/Library/Caches/pypoetry/virtualenvs/clear-J92jq7Lm-py3.11/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


,UUID,AUDIO_CLASS,COUGH_DETECTED,SNR,AGE,GENDER,RESPIRATORY_CONDITION,FEVER_MUSCLE_PAIN,STATUS,MFCC_MEAN_00,...,SPEC_SKEW_04,SPEC_SKEW_05,SPEC_SKEW_06,SPEC_SKEW_07,ZCR,SAMPLING_RATE,DURATION,MEAN_SIG,MAX_SIG,MIN_SIG
0,001d8e33-a4af-4edb-98ba-b03f891d9a6c,X,0.0306,12.713480,NaN,female,False,True,COVID-19,-409.724911,...,144.441057,14.542670,0.083621,-0.770565,0.154675,8192,9.720093,-7.622177e-06,0.246322,-0.220956
1,00357712-dd5a-4c0a-90a4-39f1f4b9d5fd,X,0.0576,0.000000,NaN,male,True,False,COVID-19,-950.545141,...,33.356659,9.232951,8.442891,-6.475617,0.307117,8192,9.960083,3.848891e-09,0.001523,-0.003599
2,009aa393-32ab-423f-8e6a-63d8d4d1476b,X,0.9782,27.690828,NaN,female,False,False,healthy,-375.274413,...,43.512556,1.649918,-8.181988,-3.324104,0.115271,8192,7.860107,2.830400e-05,0.810810,-0.797146
3,00e0a33c-6561-406e-b543-4c9f07b860f7,X,0.2798,12.695962,NaN,other,False,False,COVID-19,-459.071007,...,46.976760,-5.786418,-7.369344,5.163782,0.092737,8192,9.960083,5.392460e-05,0.546085,-0.497874
4,00f4a67a-bdce-484d-b38c-e73c84799074,X,0.0425,0.000000,NaN,female,False,False,healthy,-727.043213,...,122.650226,0.415461,18.788242,1.915282,0.053198,8192,10.005493,1.976529e-08,0.000906,-0.000890
5,014b1af2-1d84-4336-ab6b-e0e009cd86c3,X,0.8749,14.175805,NaN,female,False,False,healthy,-428.558445,...,-2.485222,22.746323,-2.303359,4.301108,0.089758,8192,9.900024,-1.646575e-06,0.854718,-0.869655
6,01674aff-9899-437f-971f-a516afec2bf5,X,0.0130,0.000000,NaN,female,False,False,healthy,-1113.408322,...,19.939339,-2.842566,-0.602072,-5.022150,0.016602,8192,9.780029,-2.242787e-08,0.001878,-0.002235
7,01a4c8d6-05df-4c24-b993-f1a9672507c5,X,0.9801,40.606809,NaN,female,False,False,healthy,-446.695161,...,-73.888767,4.145172,6.401066,4.607754,0.073889,8192,7.140015,-2.525683e-05,0.817099,-0.817167
8,01de5e23-26db-4ae9-927c-1cbaa73c0850,X,0.5271,1.476771,NaN,female,False,False,healthy,-377.189772,...,106.673957,-6.350204,4.961392,-0.901814,0.147131,8192,9.840088,-1.361638e-05,0.748618,-0.729625
9,027d8ee5-7e46-496b-a6cd-bca3476129b1,X,0.8043,31.722248,NaN,female,False,False,healthy,-560.723831,...,208.523820,-5.694691,-13.428428,-3.842737,0.059656,8192,2.400024,3.928856e-05,0.679239,-0.665882


## Salvando la info preprocesada

In [7]:
out_data_filename = root+"/preprocessed_dataset"+"/cough-classification-data_Class{}.pkl".format(iclass)
pd.to_pickle(all_data, out_data_filename)  # save df to file

Revisando que está balanceado

In [8]:
all_data[['UUID','STATUS']].groupby(['STATUS']).count().rename(columns={'UUID':'N_UUID'})

,N_UUID
STATUS,
COVID-19,189
healthy,745


In [9]:
len(spectrograms_data)

934

In [10]:
spectrograms_data.keys()

dict_keys(['001d8e33-a4af-4edb-98ba-b03f891d9a6c', '00357712-dd5a-4c0a-90a4-39f1f4b9d5fd', '009aa393-32ab-423f-8e6a-63d8d4d1476b', '00e0a33c-6561-406e-b543-4c9f07b860f7', '00f4a67a-bdce-484d-b38c-e73c84799074', '014b1af2-1d84-4336-ab6b-e0e009cd86c3', '01674aff-9899-437f-971f-a516afec2bf5', '01a4c8d6-05df-4c24-b993-f1a9672507c5', '01de5e23-26db-4ae9-927c-1cbaa73c0850', '027d8ee5-7e46-496b-a6cd-bca3476129b1', '02909ffd-16cb-47d6-9592-b30b1558227f', '02a8c9f1-f0aa-4e45-8dfb-476fabe09636', '02ae2f29-7994-4721-9061-2e32c6d10d5e', '03cad1b0-fe70-42e8-9a03-d38220320b18', '04584889-9373-497c-99ed-91e1e3aa4e01', '046405b9-ee61-4a96-84e5-6f23fe763df6', '0496f63f-f352-4f48-84e7-4eabb8acf92c', '04dc09f2-896e-438b-9509-7a0f30989bc8', '05137f19-0400-4830-aab9-569b8b0b6f52', '0526e565-ec8c-435b-b965-3c23cfb43f44', '053b59c7-d5cb-4015-8726-1fbb06f6c721', '053f46d0-4500-454e-976a-ea94bfeb778c', '054ba6b6-35c9-4b73-9016-0a132bb6d166', '05e3941b-8fda-4fb8-ab7f-4912344dac57', '061f1682-9b72-45a5-a219-04a5

In [11]:
spectrograms_data['001d8e33-a4af-4edb-98ba-b03f891d9a6c'].keys()

dict_keys(['stft', 'mfcc', 'metadata', 'features'])

In [12]:
spectrograms_data['001d8e33-a4af-4edb-98ba-b03f891d9a6c']['metadata']

{'uuid': '001d8e33-a4af-4edb-98ba-b03f891d9a6c',
 'filename': '001d8e33-a4af-4edb-98ba-b03f891d9a6c.webm',
 'duration': 10,
 'sr': 8192,
 'n_fft': 512}

In [13]:
spectrograms_data['001d8e33-a4af-4edb-98ba-b03f891d9a6c']['stft']

{'magnitude': array([[4.76835046e-04, 6.63626010e-03, 2.31311880e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.62852984e-04, 3.99090948e-03, 1.84300144e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.22849869e-04, 1.08546186e-03, 8.61110179e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [8.34304440e-08, 2.68905313e-07, 3.32211234e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.44727963e-08, 7.75485141e-08, 1.60081695e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.92900290e-08, 5.71988045e-08, 1.44992465e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 'phase': [[3.141592653589793,
   3.141592653589793,
   3.141592653589793,
   0.0,
   0.0,
   0.0,
   3.141592653589793,
   3.141592653589793,
   0.0,
   0.0,
   3.141592653589793,
   3.141592653589793,
   3.141592653589793,
   0.0,
   3.14159265358979

In [14]:
len(spectrograms_data['001d8e33-a4af-4edb-98ba-b03f891d9a6c']['features'])

110

In [15]:
for key in spectrograms_data.keys():
    print(spectrograms_data[key]['features']['STATUS'])

COVID-19
COVID-19
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
COVID-19
healthy
COVID-19
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
COVID-19
healthy
COVID-19
healthy
healthy
COVID-19
COVID-19
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
COVID-19
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
healthy
COVID-19
healthy
COVID-19
healthy
COVID-19
healthy
COVID-19
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
healthy
COVID-19
COVID-19
healthy
healthy
healthy
COVID-19
healthy
COVID-19
healthy
COVID-19
he

In [16]:
out_data_filename = root+"/preprocessed_dataset"+"/cough-classification-data_Class{}_spectrograms.pkl".format(iclass)
pd.to_pickle(spectrograms_data, out_data_filename)  

## Entrenando un modelo

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
from sklearn.metrics import classification_report
from tensorflow.keras import layers, Model, Input

In [18]:
def prepare_model_inputs(spectrograms_data):
    """
    Prepares the input data for the multimodal model from the spectrograms_data dictionary.
    
    Parameters:
    -----------
    spectrograms_data : dict
        Dictionary containing spectrogram and MFCC data for each audio file
    
    Returns:
    --------
    spectrograms : numpy.ndarray
        Array of spectrograms
    mfccs : numpy.ndarray
        Array of MFCC features
    """
    # Initialize lists to store features
    spectrograms = []
    mfccs = []
    
    # Process each audio file
    for uuid in spectrograms_data:
        # Get spectrogram magnitude
        spectrogram = np.array(spectrograms_data[uuid]['stft']['magnitude'])
        spectrograms.append(spectrogram)
        
        # Get MFCC coefficients
        mfcc = np.array(spectrograms_data[uuid]['mfcc']['coefficients'])
        mfccs.append(mfcc)
    
    # Convert to numpy arrays
    spectrograms = np.array(spectrograms)
    mfccs = np.array(mfccs)
    
    # Add channel dimension to spectrograms if needed
    if len(spectrograms.shape) == 3:
        spectrograms = spectrograms[..., np.newaxis]
    
    return spectrograms, mfccs

# Example usage:
spectrograms, mfccs = prepare_model_inputs(spectrograms_data)

# Create and compile the model
spectrogram_shape = spectrograms.shape[1:]  # (height, width, channels)
mfcc_shape = mfccs.shape[1:]  # (time_steps, n_mfcc)

In [19]:
def prepare_training_data(spectrograms_data):
    """
    Prepara los datos para el entrenamiento del modelo CNN.
    """
    # Extraer MFCCs y etiquetas
    X_mfcc = []
    X_stft = []
    y = []
    
    for uuid in spectrograms_data.keys():
        mfcc = np.array(spectrograms_data[uuid]['mfcc']['coefficients'])
        status = spectrograms_data[uuid]['features']['STATUS']
        stft = spectrograms_data[uuid]['stft']
        
        # Añadir dimensión de canal
        mfcc = np.expand_dims(mfcc, axis=-1)
        
        X_stft.append(stft)
        X_mfcc.append(mfcc)
        y.append(status)
    
    X_mfcc = np.array(X_mfcc)
    X_stft = np.array(X_stft)
    y = np.array(y)
    
    # Codificar etiquetas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_onehot = tf.keras.utils.to_categorical(y_encoded)
    
    return X_mfcc,X_stft, y_onehot, label_encoder


def create_multimodal_cnn_model(spectrogram_shape, mfcc_shape, num_classes):
    """
    Creates a multi-input CNN model that processes both spectrogram and MFCC features.
    
    Parameters:
    -----------
    spectrogram_shape : tuple
        Shape of the input spectrogram (height, width, channels)
    mfcc_shape : tuple
        Shape of the MFCC features (time_steps, n_mfcc)
    num_classes : int
        Number of output classes
    
    Returns:
    --------
    model : tf.keras.Model
        Compiled multi-input model
    """
    # Spectrogram input branch
    spectrogram_input = Input(shape=spectrogram_shape, name='spectrogram_input')
    
    # CNN layers for spectrogram processing
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(spectrogram_input)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Flatten spectrogram features
    x = layers.Flatten()(x)
    
    # MFCC input branch
    mfcc_input = Input(shape=mfcc_shape, name='mfcc_input')
    
    # Process MFCC features with a separate network
    y = layers.Conv1D(64, 3, activation='relu', padding='same')(mfcc_input)
    y = layers.BatchNormalization()(y)
    y = layers.MaxPooling1D(2)(y)
    
    y = layers.Conv1D(128, 3, activation='relu', padding='same')(y)
    y = layers.BatchNormalization()(y)
    y = layers.MaxPooling1D(2)(y)
    
    # Flatten MFCC features
    y = layers.Flatten()(y)
    
    # Concatenate both feature sets
    combined = layers.Concatenate()([x, y])
    
    # Dense layers for final classification
    z = layers.Dense(512, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    outputs = layers.Dense(num_classes, activation='softmax')(z)
    
    # Create model with both inputs
    model = Model(inputs=[spectrogram_input, mfcc_input], outputs=outputs)
    
    return model
        


def train_model(spectrograms, mfccs, y, label_encoder, batch_size=32, epochs=50):
    """
    Trains the multi-input CNN model using both spectrograms and MFCC features.
    
    This function handles the training process for our multi-input neural network,
    which processes both spectrogram and MFCC features in parallel before combining
    them for final classification.
    
    Parameters:
    -----------
    spectrograms : numpy.ndarray
        Array of spectrogram features
    mfccs : numpy.ndarray
        Array of MFCC features
    y : numpy.ndarray
        Target labels (one-hot encoded)
    label_encoder : sklearn.preprocessing.LabelEncoder
        Encoder used for the labels
    batch_size : int, optional
        Batch size for training (default: 32)
    epochs : int, optional
        Number of training epochs (default: 50)
    """
    # Split data into train/validation/test sets
    indices = np.arange(len(spectrograms))
    train_idx, temp_idx = train_test_split(indices, test_size=0.3, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)
    
    # Split spectrograms
    X_spec_train = spectrograms[train_idx]
    X_spec_val = spectrograms[val_idx]
    X_spec_test = spectrograms[test_idx]
    
    # Split MFCCs
    X_mfcc_train = mfccs[train_idx]
    X_mfcc_val = mfccs[val_idx]
    X_mfcc_test = mfccs[test_idx]
    
    # Split labels
    y_train = y[train_idx]
    y_val = y[val_idx]
    y_test = y[test_idx]
    
    # Create and compile model
    spectrogram_shape = X_spec_train.shape[1:]
    mfcc_shape = X_mfcc_train.shape[1:]
    num_classes = y_train.shape[1]
    
    # Create the multi-input model
    model = create_multimodal_cnn_model(
        spectrogram_shape=spectrogram_shape,
        mfcc_shape=mfcc_shape,
        num_classes=num_classes
    )
    
    # Compile model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        )
    ]
    
    # Train model with both inputs
    history = model.fit(
        [X_spec_train, X_mfcc_train],  # Pass both inputs as a list
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=([X_spec_val, X_mfcc_val], y_val),
        callbacks=callbacks
    )
    
    # Evaluate model
    test_loss, test_acc = model.evaluate(
        [X_spec_test, X_mfcc_test],
        y_test
    )
    print(f"\nTest accuracy: {test_acc:.4f}")
    
    # Generate predictions
    y_pred = model.predict([X_spec_test, X_mfcc_test])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    # Print classification report
    class_names = label_encoder.classes_
  # Imprimir reporte de clasificación
    class_names = label_encoder.classes_
    print("\nClassification Report:")
    print(classification_report(
        y_test_classes,
        y_pred_classes,
        target_names=class_names
    ))
    
    return model, history, {
        'test_data': ([X_spec_test, X_mfcc_test], y_test),
        'test_accuracy': test_acc,
        'predictions': y_pred,
        'true_labels': y_test_classes
    }
  

In [20]:
# Prepare the data
mfccs, spectrograms, y_onehot, label_encoder = prepare_training_data(spectrograms_data)


In [21]:
mfccs.shape

(934, 26, 161, 1)

In [22]:
spectrograms.shape

(934,)

In [23]:
# Train the model
model, history, results = train_model(
    spectrograms=spectrograms,
    mfccs=mfccs,
    y=y_onehot,
    label_encoder=label_encoder,
    batch_size=32,
    epochs=50
)

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=1. Full shape received: (None,)

In [25]:
# Guardar modelo
model.save(root+'/models/'+'model_0.keras')

## Entrenando con más datos!

In [ ]:
# Filtrar solo las clases A, B y C
metadata_filtered = metadata[metadata['audio_class'].isin(['A', 'B', 'C'])].copy()
tmp_metadata_filtered = metadata_filtered.loc[metadata_filtered['SNR']>=metadata_filtered['SNR'].quantile(0.10),]

# CONTANDO
print("Antes del remuestreo, conteo de entradas por clase STATUS en el DataFrame completo:")
tmp_metadata_count = tmp_metadata_filtered[['uuid','status']].groupby(['status']).count()
print(tmp_metadata_count)
print(len(tmp_metadata_filtered.loc[tmp_metadata_filtered['status']=='COVID-19', 'uuid'].unique()))

# redondeo
# n_resampling = int(np.ceil(tmp_metadata_count['uuid'].min()/10)*10)

# Realiza un muestreo balanceado del DataFrame usando la función sample_df_balanced
# tmp_metadata_filtered = sample_df_balanced(tmp_metadata_filtered, 'status', n_resampling)

print("\nDespués del remuestreo, conteo de entradas por clase STATUS en el DataFrame completo:")
print(tmp_metadata_filtered[['uuid','status']].groupby(['status']).count())
print(len(tmp_metadata_filtered.loc[tmp_metadata_filtered['status']=='COVID-19', 'uuid'].unique()))


### TRASNFORMANDO
all_data_abc, all_audio_metadata_abc, spectrograms_data_abc = prepare_data(input_data=tmp_metadata_filtered, audio_datadir=data_dir, sr=my_sampling_rate, 
                                            target_duration=target_duration, n_fft=my_n_fft, n_mfcc=my_n_mfcc, 
                                            fft_window_size=my_window_size, psd_freq_bins=my_psd_freqs,
                                            mfcc_feature_names=mfcc_feature_names, psd_feature_names=psd_feature_names,
                                            max_audio_samples=None, print_every_n=20, tmp_metadata = tmp_metadata_filtered) 

all_audio_metadata_abc = all_audio_metadata_abc.drop_duplicates(subset='UUID', keep='first') # this avoids spurious duplicates at the following merge

print("Merging dataframe with audio features and df with audio metadata")
all_data_abc = pd.merge(all_data_abc,all_audio_metadata_abc,on=['UUID'],how='inner')
all_data_abc.drop(['AUDIO_FILE'],axis=1,inplace=True)

#print(all_data[['UUID','STATUS']].groupby(['STATUS']).count())
print("Shape of full dataframe with features and labels: {}".format(all_data_abc.shape))
all_data_abc.head(10)


out_data_filename = root+"/preprocessed_dataset"+"/cough-classification-data_Class{}_spectrograms_abc.pkl".format(iclass)
pd.to_pickle(spectrograms_data_abc, out_data_filename)  

In [ ]:
# Preparar datos
X, y, label_encoder = prepare_training_data(spectrograms_data)

# Entrenar modelo
model, history, test_data = train_model(
X,
y,
label_encoder,
batch_size=32,
epochs=50
)

In [ ]:
model = create_multimodal_cnn_model(spectrogram_shape, mfcc_shape, num_classes=your_num_classes)

# Compile the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Train the model
history = model.fit(
    [spectrograms, mfccs],  # Note: now we pass a list of inputs
    labels,
    batch_size=32,
    epochs=50,
    validation_split=0.2
)